In [ ]:
!pip install timm

In [ ]:
import os
import json
from tqdm.notebook import tnrange, tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
import cv2


import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torch.nn as nn

from timm import create_model

device = torch.device("cpu") if not torch.cuda.is_available() else torch.device("cuda:0")
print("Using device", device)

In [ ]:
model_name = "vit_base_patch16_224"
# create a ViT model : https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py
model = create_model(model_name, pretrained=True).to(device)

In [ ]:
# Define transforms for test
IMG_SIZE = (224, 224)
NORMALIZE_MEAN = (0.5, 0.5, 0.5)
NORMALIZE_STD = (0.5, 0.5, 0.5)
transforms = [
              T.Resize(IMG_SIZE),
              T.ToTensor(),
              T.Normalize(NORMALIZE_MEAN, NORMALIZE_STD)
              ]

transforms = T.Compose(transforms)

In [ ]:
%%capture
# ImageNet Labels
!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
imagenet_labels = dict(enumerate(open('ilsvrc2012_wordnet_lemmas.txt')))

# Demo Image
!wget https://github.com/hirotomusiker/schwert_colab_data_storage/blob/master/images/vit_demo/santorini.png?raw=true -O santorini.png
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/Senior_Airman_Benton_Pohlman_fires_an_M4_carbine_rifle_%2834135723246%29.jpg/1200px-Senior_Airman_Benton_Pohlman_fires_an_M4_carbine_rifle_%2834135723246%29.jpg -O assault_rifle.jpg
img = PIL.Image.open('santorini.png')
img_tensor = transforms(img).unsqueeze(0).to(device)

#quick test
output = model(img_tensor)
print(imagenet_labels[int(torch.argmax(output))])

In [ ]:
TENSOR_MEANS, TENSOR_STD = torch.FloatTensor(NORMALIZE_MEAN)[:,None,None], torch.FloatTensor(NORMALIZE_STD)[:,None,None]
def patch_forward(patch):
    # Map patch values from [-infty,infty] to min and max
    patch = (torch.tanh(patch) + 1 - 2 * TENSOR_MEANS) / (2 * TENSOR_STD)
    return patch

def place_patch(img, patch):
    for i in range(img.shape[0]):
        temp = img[i]
        h_offset = np.random.randint(0,img.shape[2]-patch.shape[1]-1)
        w_offset = np.random.randint(0,img.shape[3]-patch.shape[2]-1)
        temp[:, h_offset:h_offset+patch.shape[1],w_offset:w_offset+patch.shape[2]] = patch_forward(patch)
        img[i] = temp
    return img


In [ ]:
def patch_attack(input_image_path, resize_input_image, model, target_class, patch_size, num_epochs, pretrained_patch):
    
    # Create parameter and optimizer
    patch_size = (patch_size, patch_size)
    if pretrained_patch is None:
        patch = nn.Parameter(torch.zeros(3, patch_size[0], patch_size[1]), requires_grad=True)
    else:
        patch_np = np.load(pretrained_patch)
        patch = nn.Parameter(torch.tensor(patch_np), requires_grad=True)

    optimizer = torch.optim.SGD([patch], lr=2e-1, momentum=0.8)
    loss_module = nn.CrossEntropyLoss()

    input_pil_image = Image.open(input_image_path)
    if resize_input_image:
        input_pil_image = input_pil_image.resize((224,224), Image.NEAREST)
    input_tensor = tensor_transform(input_pil_image)
    input_tensor = torch.transpose(input_tensor, 2, 1)

    print(output[0][target_class])

    # # Training loop
    for epoch in tnrange(num_epochs):

        batch =  (input_tensor.unsqueeze(0)).repeat_interleave(16, dim = 0)
        patched_batch = place_patch(batch, patch)
        pred = model(patched_batch.to(device))
        labels = torch.zeros(patched_batch.shape[0], device=pred.device, dtype=torch.long).fill_(target_class)
        loss = loss_module(pred, labels)
        optimizer.zero_grad()
        loss.mean().backward()
        optimizer.step()

    patch_temp=patch.detach().cpu().permute(2, 1, 0)
    patch_temp = (torch.tanh(patch_temp) + 1) / 2
    patch_temp = np.clip(patch_temp, a_min=0.0, a_max=1.0)
    plt.imshow(patch_temp)
    plt.grid(False)
    plt.show()
    print("op/ on target class <beta>", pred[0][target_class])

    return(patch)

In [ ]:
plt.rcParams["figure.figsize"] = (5,5)
final_patch_tensor = patch_attack(input_image_path='assault_rifle.jpg',
                                  resize_input_image = True,
                                  model = model, 
                                  target_class = 605,
                                  patch_size = 32,
                                  num_epochs = 200,
                                  pretrained_patch=None)

In [ ]:
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def attack_image(input_tensor, patch_tensor):
    batch =  (input_tensor.unsqueeze(0)).repeat_interleave(1, dim = 0)
    patched_batch = place_patch(batch, patch_tensor)
    return patched_batch[0]

In [ ]:
def top_5_predictions(input_image_path, resize_input_image, patch_tensor, model, label_names):

    input_pil_image = Image.open(input_image_path)
    if resize_input_image:
        input_pil_image = input_pil_image.resize((224,224), Image.NEAREST)
    tensor_transform = T.ToTensor()
    input_tensor = tensor_transform(input_pil_image)
    input_tensor = torch.transpose(input_tensor, 2, 1)

    input_show = np.empty_like(input_tensor.permute(2,1,0))
    input_show[:] = input_tensor.permute(2,1,0)

    fig, ax = plt.subplots(nrows=2, ncols=2)
    ax[0][0].imshow(input_show)
    ax[0][0].set_xlabel('Input')

    attacked = attack_image(input_tensor, patch_tensor)

    ax[0][1].imshow(attacked.permute(2,1,0))
    ax[0][1].set_xlabel('Adversarial Input')

    input_1 = input_tensor.unsqueeze(0).type(torch.cuda.FloatTensor).to(device)
    input_2 = attacked.unsqueeze(0).type(torch.cuda.FloatTensor).to(device)

    with torch.no_grad():
        results_1 = torch.softmax(model(input_1), dim=-1).cpu()[0]
        results_2 = torch.softmax(model(input_2), dim=-1).cpu()[0]
        values_1, indices_1 = results_1.topk(5)
        values_2, indices_2 = results_2.topk(5)

    argmax_1, argmax_2 = torch.argmax(results_1), torch.argmax(results_2)
    names_1, names_2 = label_names[indices_1.cpu().numpy()], label_names[indices_2.cpu().numpy()]

    ax[1][0].bar([i for i in range(5)], values_1*100, color=['green', 'navy', 'navy','navy', 'navy'])
    ax[1][0].set_ylim([0, 100])
    ax[1][0].set_xticks([i for i in range(5)])
    ax[1][0].set_xticklabels(names_1, rotation=90)
    ax[1][0].set_xlabel('Classifier Output')
    ax[1][0].set_ylabel('Confidence')
    ax[1][0].grid()
    
    ax[1][1].bar([i for i in range(5)], values_2*100, color=['red', 'navy', 'navy','navy', 'navy'])
    ax[1][1].set_ylim([0, 100])
    ax[1][1].set_xticks([i for i in range(5)])
    ax[1][1].set_xticklabels(names_2, rotation=90)
    ax[1][1].set_xlabel('Classifier Output')
    ax[1][1].set_ylabel('Confidence')
    ax[1][1].grid()


In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
labels_np = np.array(list(imagenet_labels.values()))
top_5_predictions(input_image_path = 'assault_rifle.jpg', 
                  resize_input_image = True, 
                  patch_tensor = final_patch_tensor.detach(), 
                  model = model, 
                  label_names = labels_np)